In [1]:
CONFIG_FILE_NAME = '../config/file_config_air.yaml'
LOG_FILE_NAME = '../data/categorization_pipeline.log'

In [2]:
from src.format_validators import validate_config
from src.loaders import load_config, construct_config
from src.logger import set_up_logger
from src.statement_processing_pipeline import StatementProcessor

In [3]:
# set up logging & metadata
stream_logs = set_up_logger(LOG_FILE_NAME)
cf = load_config(CONFIG_FILE_NAME)

INFO:root:Loaded config from :../config/file_config_air.yaml
INFO:root:{'file_name_input': '../data/raw/airbank_1723805018_2025-07-03_16-27.csv', 'file_name_output': '../data/processed/air_2025_01_06.xlsx', 'fix_file_name': '../data/processed/test.json', 'input_config': '../config/generic/input_config_airbank.yaml', 'output_config': '../config/generic/output_config.yaml', 'keywords_config': '../config/generic/keywords.yaml'}


In [4]:
# assemble full config for the pipeline
if validate_config(config_dict=cf, cf_type='High-level') and validate_config(config_dict=construct_config(cf), cf_type='Full'):
    config = construct_config(cf)

INFO:root:Loaded config from :../config/generic/input_config_airbank.yaml
INFO:root:{'parsing': {'delimiter': ';', 'encoding': 'utf-8'}, 'input_format': {'payment_category': 'Kategorie plateb', 'date': {'col': 'Datum provedení', 'format': ['%d/%m/%Y']}, 'counterparty_ids': ['Obchodní místo', 'Název protistrany', 'Název účtu protistrany', 'Číslo účtu protistrany', 'Poznámka k úhradě'], 'message': ['Poznámka pro mne', 'Zpráva pro příjemce'], 'amount': {'col': 'Částka v měně účtu', 'format': {'decimal_separator': ',', 'thousands_separator': ''}}, 'transaction_type': 'Typ úhrady', 'inbound_keyword': 'Příchozí'}}
INFO:root:Loaded config from :../config/generic/output_config.yaml
INFO:root:{'display_groups': 'Kategorie_plateb', 'counterparty': 'counterparty', 'message': 'message', 'amount': 'částka', 'date': {'col': 'datum', 'format': ['%Y-%m-%d']}, 'generated_suggestions': 'category_suggestion', 'transaction_type': 'Typ transakce', 'payment_category': 'Kategorie_plateb', 'categories_excl_fr

In [5]:
# instantiate the pipeline
processor = StatementProcessor(config, stream_logs)

In [6]:
# first part of the pipeline results in a file user should edit
processor.run_first_part()

INFO:root:Running step LOAD
INFO:root:Loaded CSV from :../data/raw/airbank_1723805018_2025-07-03_16-27.csv
INFO:root:    Datum provedení Směr úhrady       Typ úhrady Kategorie plateb Měna účtu  \
0        30/06/2025     Odchozí    Platba kartou           Zábava       CZK   
1        27/06/2025     Odchozí    Platba kartou          Doprava       CZK   
2        27/06/2025     Odchozí    Platba kartou          Doprava       CZK   
3        26/06/2025     Odchozí   Odchozí úhrada    nezapočítávat       CZK   
4        26/06/2025     Odchozí   Odchozí úhrada    nezapočítávat       CZK   
..              ...         ...              ...              ...       ...   
148      13/01/2025     Odchozí    Platba kartou           Nákupy       CZK   
149      12/01/2025     Odchozí    Platba kartou           Nákupy       CZK   
150      11/01/2025     Odchozí    Platba kartou           Nákupy       CZK   
151      06/01/2025    Příchozí  Příchozí úhrada  Příchozí úhrada       CZK   
152      04/01

,Typ transakce,Kategorie_plateb,částka,datum,counterparty,message,month,category_suggestion
0,Platba kartou,Zábava,-199.00,2025-06-30,"Google Play Apps Sir John Rogerson's Quay, Dub...",nan,202506,[]
1,Platba kartou,Doprava,-1.00,2025-06-27,"CSAD Frydek Mistek Frydek-Mistek, 000__ČSAD Uh...",nan,202506,Doprava
2,Platba kartou,Doprava,-1.00,2025-06-27,"KODIS.cz Ostrava, 000__Koordinátor ODIS s.r.o....",nan,202506,Doprava
3,Odchozí úhrada,nezapočítávat,-10000.00,2025-06-26,nan__Jan Krejčí__1015274383/5500,nan__Hypo,202506,nezapočítávat
4,Odchozí úhrada,nezapočítávat,-150000.00,2025-06-26,nan__Jan Krejčí__1015274383/5500,nan__Hypo,202506,nezapočítávat
...,...,...,...,...,...,...,...,...
148,Platba kartou,Nákupy,-247.30,2025-01-13,"Billa Sokolovska 14 Sokolovska, Praha - Metro,...",nan,202501,Nákupy
149,Platba kartou,Nákupy,-1043.03,2025-01-12,"Penny U Stare posty 8 U Stare posty, Praha - B...",nan,202501,Nákupy
150,Platba kartou,Nákupy,-130.30,2025-01-11,"Penny U Stare posty 8 U Stare posty, Praha - B...",nan,202501,Nákupy
151,Příchozí úhrada,Příchozí úhrada,1600.00,2025-01-06,nan__Polina Galueva__3521606001/5500,nan,202501,sport


In [21]:
# after changes to categorization in the file are done, this command should be run
processor.run_second_part()

In [ ]:
# verify data with categorization
processor.get_final_data()

In [ ]:
# generate statistics
processor.generate_aggregate_data()

In [ ]:
#save output
processor.save_output()